In [0]:
import numpy as np
import tensorflow as tf
from sklearn.decomposition import PCA
import scipy.io
from sklearn.model_selection import train_test_split
import keras
from keras import optimizers
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
import math
from scipy import stats

Using TensorFlow backend.


In [0]:
id = "1_ZtXbVbjQqC3Y-O04Jld9O0nXrbuAq01"
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('SUB_100307_S2.mat')  
data = scipy.io.loadmat('SUB_100307_S2.mat')

In [0]:
data

{'D': array([[[-2.09151336e-14, -5.41053004e-14,  6.54931967e-14, ...,
           1.24200547e-13, -1.96418518e-13, -5.45248851e-14],
         [-2.21179479e-13, -3.99130518e-13,  1.68847703e-13, ...,
           3.17905311e-14, -1.90115749e-13, -2.19864366e-13],
         [-1.93471751e-14, -1.34969500e-13,  2.55453833e-13, ...,
          -1.45627658e-13, -2.63070784e-13, -4.70834048e-14],
         ...,
         [ 2.73748297e-13,  9.00729087e-14,  1.04738340e-13, ...,
          -2.42499423e-13,  1.23867057e-13,  1.33588873e-13],
         [ 2.02379935e-13, -2.16917027e-13, -1.64777102e-13, ...,
          -3.31792702e-13,  2.00359854e-13, -2.72261635e-13],
         [-3.66881564e-14, -8.93052808e-14,  8.01718894e-14, ...,
          -1.85836754e-13,  3.06366502e-14, -1.60810714e-13]],
 
        [[ 1.98385669e-13, -7.18277038e-14,  4.60407179e-15, ...,
          -1.97723751e-13,  1.78413619e-13,  1.91885171e-13],
         [ 7.01846943e-14, -2.02696918e-13, -2.00668416e-14, ...,
          -3.248

In [0]:
data['time_vect'].shape, data['D'].shape

((1, 2035), (240, 2035, 124))

In [0]:
x =  data['D']
x = np.reshape(x,(x.shape[0],x.shape[1]*x.shape[2]))
x = np.transpose(x)

**By using PCA**

In [0]:
X = stats.zscore(x,axis=1)
pca = PCA(0.95)
mu = np.mean(X, axis=0)
Xhat = pca.fit_transform(X)
reconstructionX = np.dot(Xhat, pca.components_)

In [0]:
math.sqrt(np.mean((reconstructionX - X)**2))

0.22329215035176186

**By using auto encoders**

In [0]:
epochs = 50
verbose = 1
train_validation_split = 0.2

In [0]:
input_dims = X.shape[1]
model = keras.Sequential([
      keras.layers.Dense(Xhat.shape[1],input_dim = input_dims,activation = tf.nn.relu),
      keras.layers.Dense(X.shape[1], activation = 'sigmoid')
    ])
model.summary()
sgd = optimizers.RMSprop(lr=1e-1)
model.compile(optimizer='adam', loss='mse',metrics=['accuracy'])
history = model.fit(X,X,epochs=epochs,validation_split=train_validation_split,verbose=verbose,shuffle=True)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 85)                20485     
_________________________________________________________________
dense_8 (Dense)              (None, 240)               20640     
Total params: 41,125
Trainable params: 41,125
Non-trainable params: 0
_________________________________________________________________
Train on 201872 samples, validate on 50468 samples
Epoch 1/50
201872/201872 [==============================] - 28s 138us/step - loss: 0.6314 - acc: 0.3214 - val_loss: 0.6030 - val_acc: 0.4376
Epoch 2/50
  4256/201872 [..............................] - ETA: 24s - loss: 0.6060 - acc: 0.4617

In [0]:
rmse = math.sqrt(model.evaluate(X,X)[1])

In [0]:
rmse